# Projeto 1 - Ciência dos Dados

Nome: Arthur Fuhrken

Nome: Pedro Mahfuz

Nome: Rodrigo Furukawa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import emoji

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\osbor\Documents\2021.1\Cdados DP\Projeto 1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'ruffles.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Valor
0,uma criancinha deu um ruffles pra ela,1
1,rt @favskyliex: 📖 sorteio 📖\n\npara concorrer ...,0
2,"@_adryelleac @americanascom aí sim, adry! imag...",2
3,@b_ruffles eu vo para de estuda também,0
4,assimmm completamente eu arrumando as parede d...,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Valor
0,@vulgo_ruffles amo vc 🥰🥰,0
1,rt @igoritonobrasil: 🏐💕 amor doce 💕🏐\n#haikyuu...,0
2,rt @bonecker_: incrível que eu taco o fodase p...,2
3,"[9:55 pm, 17/03/2021] alexia: eu comi só ruffl...",1
4,hj não foi um bom dia,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é a famosa batata "Ruffles", consideramos como tweets relevantes os que falassem do produto de uma maneira pejorativa ou positiva. Consideramos como irrelevantes, os tweets que falavam do produto, mas não agregavam nada a ele. Por fim, consideramos muito irrelevantes os tweets que não faziam qualquer tipo de menção ao produto ou que continham conteúdo completamente desagregado ao valor dele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#Criando classes "cleanup", usadas para deixar as strings mais padronizadas.

def cleanup_pontuacao(tweets):
    p = '[!-.:?;"\n"()''"",_%$®|=“”—/]'
    padrao = re.compile(p)
    remove_ponto = re.sub(padrao,' ',tweets)
    espaco = ' '
    padrao = re.compile(espaco)
    tweets_espaco = re.sub(padrao, ' ', remove_ponto)
    return tweets_espaco
    
def cleanup_espacos(tweets):    
    espacos = ' '
    padrao = re.compile(espacos)
    remove_espaco = re.sub(padrao,' ', tweets)
    return remove_espaco

def cleanup_link(tweets):
    link = r'http[^\s]*'
    padrao = re.compile(link)
    remove_link = re.sub(padrao,'',tweets)
    return remove_link

trein_str=str()
for n in train['Treinamento']:
    trein_str+=' '+n



trein = trein_str.lower()
trein1 = cleanup_pontuacao(trein)
trein2 = cleanup_espacos(trein1)
trein_limpo = cleanup_link(trein2)
trein_limpo

' uma criancinha deu um ruffles pra ela rt @favskyliex  📖 sorteio 📖  para concorrer um box de livros até r  250 siga as regras     seguir @favskyliex   @aeiinglthyns   @booksspace… @ adryelleac @americanascom aí sim  adry  imagina comprar uma ruffles  tira onda por apenas 0 99 centavos  isso é coisa de outro mundo    😝🌀 @b ruffles eu vo para de estuda também assimmm completamente eu arrumando as parede do quarto       t co 7uvnmaaafh o jeitoq eu comi só metade de um pacote de batata ruffles e minha glicose foi de 85 pra 110 eu quero me m nao aguento mais a androginia quero ser bin4r14 ruffles de churrasco e uma coca bem geladinha é tudo pra mim é só em mim que o salgadinho ruffles da dor de cabeça  sempre escolho ruffles ou sensações    t co 1auiismgcq rt @miyadzu   o mlk de vila agora feliz voltará para sua mansão pra comer ruffles com tampico e mascar um ping pong de sobremesa 🤝 @m4rodri nossa eu parei no ruffles  n aguentei  olha a manteia no inicio do video mano pqp do nada mo vont

In [7]:
#Usando UNICODE emoji para separar os emojis

def seleciona_emoji(character):
        return character in emoji.UNICODE_EMOJI 
    
def character_tem_emoji(texto):
    return ''.join(' ' + char + ' ' if seleciona_emoji(char) else char for char in texto).strip()    

trein_f = character_tem_emoji(trein_limpo)

In [8]:
#Criando um pd.Series com todas as palavras e suas frequencias relativas

palavras = trein_f.split()

total = pd.Series(palavras)
total.value_counts(True)

ruffles       0.046613
de            0.036663
e             0.025896
que           0.018400
a             0.017582
                ...   
nunk          0.000136
choras        0.000136
batatas       0.000136
trio          0.000136
@fwrdnanda    0.000136
Length: 2186, dtype: float64

In [9]:
train['Valor']=train['Valor'].astype('category')
train['Valor'].cat.categories = ["Muito Irrelevante","Irrelevante","Relevante"]

train['Valor'].value_counts(True)

Muito Irrelevante    0.396
Irrelevante          0.330
Relevante            0.274
Name: Valor, dtype: float64

In [10]:
filtro_0 = train['Valor'] == 'Muito Irrelevante'
muito_irrelevante = train.loc[filtro_0,:]

filtro_1 = train['Valor'] == 'Irrelevante'
irrelevante = train.loc[filtro_1,:]

filtro_2 = train['Valor'] == 'Relevante'
relevante = train.loc[filtro_2,:]


In [14]:
#Limpando relevantes da tabela Treinamento com as funções criadas no inicio

relevantes_clean=str()
for n in relevante['Treinamento']:
    relevantes_clean+=' '+n
    
relevantes = relevantes_clean.lower()
relevantes1 = cleanup_pontuacao(relevantes)
relevantes2 = cleanup_espacos(relevantes1)
relevantes_limpo = cleanup_link(relevantes2)
relevantes_emoji = character_tem_emoji(relevantes_limpo)

palavras1 = relevantes_emoji.split()
total1 = pd.Series(palavras1)

In [15]:
#Limpando irrelevantes da tabela Treinamento com as funções criadas no inicio

irrelevantes_clean=str()
for n in irrelevante['Treinamento']:
    irrelevantes_clean+=' '+n
    
irrelevantes = irrelevantes_clean.lower()
irrelevantes1 = cleanup_pontuacao(irrelevantes)
irrelevantes2 = cleanup_espacos(irrelevantes1)
irrelevantes_limpo = cleanup_link(irrelevantes2)
irrelevantes_emoji = character_tem_emoji(irrelevantes_limpo)

palavras2 = irrelevantes_emoji.split()
total2 = pd.Series(palavras2)

In [16]:
#Limpando m_irrelevantes da tabela Treinamento com as funções criadas no inicio

m_irrelevantes_clean=str()
for n in muito_irrelevante['Treinamento']:
    m_irrelevantes_clean+=' '+n
    
m_irrelevantes = m_irrelevantes_clean.lower()
m_irrelevantes1 = cleanup_pontuacao(m_irrelevantes)
m_irrelevantes2 = cleanup_espacos(m_irrelevantes1)
m_irrelevantes_limpo = cleanup_link(m_irrelevantes2)
m_irrelevantes_emoji = character_tem_emoji(m_irrelevantes_limpo)

palavras3 = m_irrelevantes_emoji.split()
total3 = pd.Series(palavras3)

In [28]:
#Criando lista com palavras limpas relevantes

relevantes = []

for i in relevante['Treinamento']:
    relevantes_clean = i.lower()
    relevantes1 = cleanup_pontuacao(relevantes_clean)
    relevantes2 = cleanup_espacos(relevantes1)
    relevantes_limpo = cleanup_link(relevantes2)
    relevantes_emoji = character_tem_emoji(relevantes_limpo)
    
    for t in relevantes_emoji.split():
        relevantes.append(t)
        

 

In [30]:
#Criando lista com palavras limpas irrelevantes

irrelevantes = []

for i in irrelevante['Treinamento']:
    irrelevantes_clean = i.lower()
    irrelevantes1 = cleanup_pontuacao(irrelevantes_clean)
    irrelevantes2 = cleanup_espacos(irrelevantes1)
    irrelevantes_limpo = cleanup_link(irrelevantes2)
    irrelevantes_emoji = character_tem_emoji(irrelevantes_limpo)
    
    for t in irrelevantes_emoji.split():
        irrelevantes.append(t)
        


In [33]:
#Criando lista com palavras limpas muito irrelevantes

m_irrelevantes = []

for i in muito_irrelevante['Treinamento']:
    m_irrelevantes_clean = i.lower()
    m_irrelevantes1 = cleanup_pontuacao(m_irrelevantes_clean)
    m_irrelevantes2 = cleanup_espacos(m_irrelevantes1)
    m_irrelevantes_limpo = cleanup_link(m_irrelevantes2)
    m_irrelevantes_emoji = character_tem_emoji(m_irrelevantes_limpo)
    
    for t in m_irrelevantes_emoji.split():
        m_irrelevantes.append(t)


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**